In [1]:
!pip install tensorflow
!pip install keras


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# read text sequence dataset
train_seq_df = pd.read_csv("datasets/train/train_text_seq.csv")
train_seq_X = train_seq_df['input_str'].tolist()
train_seq_Y = train_seq_df['label'].tolist()

# read text sequence dataset
valid_seq_df = pd.read_csv("datasets/valid/valid_text_seq.csv")
valid_seq_X = valid_seq_df['input_str'].tolist()
valid_seq_Y = valid_seq_df['label'].tolist()

test_seq_X = pd.read_csv("datasets/test/test_text_seq.csv")['input_str'].tolist()

X= train_seq_X
y = train_seq_Y

In [4]:
# Character-level tokenizer for emoticons
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(X)

# Convert emoticons into sequences of integers
X_sequences = tokenizer.texts_to_sequences(X)

# Padding to ensure all sequences are of the same length
max_len = max(len(seq) for seq in X_sequences)
X_padded = pad_sequences(X_sequences, maxlen=max_len, padding='post')


In [5]:
# One-hot encode the integer sequences (each token becomes a one-hot vector)
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because Keras indexing starts from 1
X_one_hot = tf.keras.utils.to_categorical(X_padded, num_classes=vocab_size)


In [6]:
# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_one_hot, y_encoded, test_size=0.2, random_state=42)


In [53]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Build the LSTM model with reduced parameters
model = Sequential()

# Reduce the LSTM units to 64 (fewer parameters)
model.add(LSTM(44, input_shape=(max_len, vocab_size), recurrent_dropout=0.2))

# Dropout layer to prevent overfitting
model.add(Dropout(0.3))  # Reduce dropout to balance performance and regularization

# Dense layer: the output layer with softmax activation for classification
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary to see the parameter count
model.summary()


Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_18 (LSTM)                  │ (None, 44)             │         9,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 44)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 2)              │            90 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,946 (38.85 KB)

 Trainable params: 9,946 (38.85 KB)

 Non-trainable params: 0 (0.00 B)

In [58]:
# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=12, validation_data=(X_val, y_val))


Epoch 1/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.7179 - loss: 0.5408 - val_accuracy: 0.7528 - val_loss: 0.5158
Epoch 2/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.7230 - loss: 0.5357 - val_accuracy: 0.7542 - val_loss: 0.5081
Epoch 3/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.7310 - loss: 0.5298 - val_accuracy: 0.7585 - val_loss: 0.5033
Epoch 4/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.7214 - loss: 0.5415 - val_accuracy: 0.7578 - val_loss: 0.5047
Epoch 5/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.7375 - loss: 0.5179 - val_accuracy: 0.7549 - val_loss: 0.4937
Epoch 6/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.7360 - loss: 0.5189 - val_accuracy: 0.7648 - val_loss: 0.4869
Epoch 7/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.7510 - loss: 0.5084 - val_accuracy: 0.7719 - val_loss: 0.4774
Epoch 8/30
472/472 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.7329 - loss: 0.5199 - val_accu

In [59]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy * 100:.4f}")


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8642 - loss: 0.3375
Validation Accuracy: 84.2514
